In [2]:
import numpy as np
import pandas as pd
import random
import math 
import scipy.stats as st
import warnings
import sklearn as sk
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [3]:
warnings.simplefilter(action='ignore', category=Warning)
randstates = [random.randint(1,200) for _ in range(5)]

In [4]:
full = pd.read_stata('tedsd_puf_2017_edited.dta')
full = full[full['DSMCRIT']==5]
full = full[full['SERVICES']>5]
df = full.take(np.random.permutation(len(full))[:100000])
df.to_csv('training_permutation.csv')

In [5]:
df = pd.read_csv('training_permutation.csv')
df.drop(columns=['DSMCRIT', 'DISYR', 'CASEID', 'CBSA2010', 'DETNLF', 'DETNLF_D', 
                 'DIVISION', 'STFIPS', 'MARSTAT', 'PRIMPAY', 'IDU', 'DETCRIM', 'ROUTE2',
                 'ROUTE3', 'FREQ2', 'FREQ3', 'FREQ1_D', 'FREQ2_D', 'FREQ3_D', 'HLTHINS',
                 'EMPLOY_D', 'SERVICES', 'SERVICES_D', 'FRSTUSE2', 'FRSTUSE3'], inplace=True)
df.replace(to_replace = -9, value = np.NaN, inplace = True)
df.dropna(subset=['NOPRIOR', 'RACE', 'ETHNIC', 'ARRESTS', 'EDUC', 'EMPLOY', 
                  'PSOURCE', 'METHUSE', 'GENDER', 'LIVARAG', 'FRSTUSE1', 'ROUTE1', 'FREQ1', 'SUB1'], inplace=True)
df.index = range(len(df))

In [6]:
drugs_recode = {1:1, 2:2, 3:3, 4:4, 5:5, 6:5, 7:5, 8:8, 9:8, 10:6, 11:6, 12:8, 13:7, 14:8, 15:8, 16:8, 17:8, 18:8, 19:8}
df[['SUB1', 'SUB2', 'SUB3', 'SUB1_D', 'SUB2_D', 'SUB3_D']] = df[
    ['SUB1', 'SUB2', 'SUB3', 'SUB1_D', 'SUB2_D', 'SUB3_D']].replace(drugs_recode)

ethnic_recode = {1:1, 2:1, 3:1, 4:0, 5:1}
df['ETHNIC'] = df['ETHNIC'].replace(ethnic_recode)

reasons_recode = {2:0, 3:0, 4:0,5:0, 6:0, 7:0}
df['REASON'] = df['REASON'].replace(reasons_recode)

df['METHUSE'] = df['METHUSE'].replace({2:0})
df['PREG'] = df['PREG'].replace({np.NaN:0, 2:0})
df['GENDER'] = df['GENDER'].replace({2:0})
df['VET'] = df['VET'].replace({2:0})
df['PSYPROB'] = df['PSYPROB'].replace({2:0})

In [7]:
y = df['REASON']
df.drop(columns=['REASON', 'Unnamed: 0'],inplace=True)
X = df

In [ ]:
print(X.columns)

In [26]:
race = ['ALASKA_NAT', 'AM_INDIAN', 'API', 'BLACK', 'WHITE', 'ASIAN', 'ONE_R', 'TWO_R', 'HAWAIIAN']
livarr = ['HOMELESS', 'DEP_LIV', 'INDEP_LIV']
livarr_d = ['LIVARR_D_MISSING', 'HOMELESS_D', 'DEP_LIV_D', 'INDEP_LIV_D']
sources = ['INDIV', 'ALC_DRUG_CAREPROV', 'OTHER_CAREPROV', 'SCHOOL', 'EMPLOYER', 'COMM_REF', 'CRIM_JUST']
subs1 = ['ALC1', 'COC1', 'MARIJ1', 'HEROIN_OPS1', 'METH1', 'BENZ1', 'OTHER1']
subs2 = ['SUBS2_MISSING', 'NONE2', 'ALC2', 'COC2', 'MARIJ2', 'HEROIN_OPS2', 'METH2', 'BENZ2', 'OTHER2']
subs3 = ['SUBS3_MISSING', 'NONE3', 'ALC3', 'COC3', 'MARIJ3', 'HEROIN_OPS3', 'METH3', 'BENZ3', 'OTHER3']
subs1_d = ['SUBS1_D_MISSING', 'NONE1_D', 'ALC1_D', 'COC1_D', 'MARIJ1_D', 'HEROIN_OPS1_D', 'METH1_D', 'BENZ1_D', 'OTHER1_D']
subs2_d = ['SUBS2_D_MISSING', 'NONE2_D', 'ALC2_D', 'COC2_D', 'MARIJ2_D', 'HEROIN_OPS2_D', 'METH2_D', 'BENZ2_D', 'OTHER2_D']
subs3_d = ['SUBS3_D_MISSING', 'NONE3_D', 'ALC3_D', 'COC3_D', 'MARIJ3_D', 'HEROIN_OPS3_D', 'METH3_D', 'BENZ3_D', 'OTHER3_D']
regions = ['US_TERR', 'NE', 'MW', 'SOUTH', 'WEST']
ad_ind = ['ALC_ONLY', 'DRUGS_ONLY', 'ALC_DRUGS']
jobs = ['FT', 'PT', 'UNEMP', 'NLF']
routes1 = ['ORAL1', 'SMOK1', 'INHAL1', 'INJ1', 'OTHER_ROUTE1']
inc = ['INC_MISSING', 'WAGE', 'PUB_ASSIST', 'RET_DIS', 'OTHER', 'NONE']

onehot_cols = [regions, sources, livarr, livarr_d, race, subs1, subs2, subs3, subs1_d, subs2_d, subs3_d, ad_ind, jobs, routes1, inc]

In [27]:
def ML_rf(X,y,random_state,n_folds):
    # create a test set
    global onehot_cols 
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state, stratify=y)

    # splitter for _other
    skf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)

    # create the pipeline: preprocessor + supervised ML method
    cat_cols = ['REGION', 'PSOURCE', 'LIVARAG', 'LIVARAG_D', 'RACE', 
                'SUB1', 'SUB2', 'SUB3', 'SUB1_D', 'SUB2_D', 'SUB3_D', 'ALCDRUG', 'EMPLOY', 'ROUTE1', 'PRIMINC']
    
    cont_cols = ['EDUC', 'LOS', 'ARRESTS', 'ARRESTS_D', 'FREQ1', 'FRSTUSE1', 'AGE', 'ETHNIC', 'GENDER', 
                 'NOPRIOR', 'PREG', 'DAYWAIT', 'FREQ_ATND_SELF_HELP', 'FREQ_ATND_SELF_HELP_D', 
                 'METHUSE', 'VET', 'PSYPROB']
    
    categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(missing_values = np.NaN, strategy='constant', fill_value = 0)),
            ('onehot', OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])
    
    other_transformer = Pipeline(steps=[
            ('imputer2', IterativeImputer(missing_values = np.NaN, 
                                          estimator=RandomForestRegressor(), random_state=random_state))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cont', other_transformer, cont_cols),
            ('cat', categorical_transformer, cat_cols)])
    scaler = StandardScaler()
    pipe = make_pipeline(preprocessor, scaler, RandomForestClassifier(random_state=random_state))

    # the parameter(s) we want to tune
    param_grid = {'randomforestclassifier__max_depth': list(range(2, 8)),
                 'randomforestclassifier__min_samples_split': list(range(2, 10))}

    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score,greater_is_better=True),
                        cv=skf, return_train_score = True, iid=True)
    # do kfold CV on _other
    grid.fit(X_other, y_other)
    #print(grid.cv_results_)
    return grid, X_test, y_test

In [31]:
def ML_log(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state, stratify=y)

    # splitter for _other
    skf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)

    # create the pipeline: preprocessor + supervised ML method
    cat_cols = ['REGION', 'PSOURCE', 'LIVARAG', 'LIVARAG_D', 'RACE', 
                'SUB1', 'SUB2', 'SUB3', 'SUB1_D', 'SUB2_D', 'SUB3_D', 'ALCDRUG', 'EMPLOY', 'ROUTE1', 'PRIMINC']
    
    cont_cols = ['EDUC', 'LOS', 'ARRESTS', 'ARRESTS_D', 'FREQ1', 'FRSTUSE1', 'AGE', 'ETHNIC', 'GENDER', 
                 'NOPRIOR', 'PREG', 'DAYWAIT', 'FREQ_ATND_SELF_HELP', 'FREQ_ATND_SELF_HELP_D', 
                 'METHUSE', 'VET', 'PSYPROB']
    
    categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(missing_values = np.NaN, fill_value = 0, strategy='constant')),
            ('onehot', OneHotEncoder(categories='auto', sparse=False,handle_unknown='ignore'))])
    
    other_transformer = Pipeline(steps=[
            ('imputer2', IterativeImputer(missing_values = np.NaN, 
                                          estimator=RandomForestRegressor(), random_state=random_state))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, cat_cols),
            ('cont', other_transformer, cont_cols)])
    scaler = StandardScaler()
    pipe = make_pipeline(preprocessor, scaler, LogisticRegression(solver='saga', max_iter=1000))

    # the parameter(s) we want to tune
    param_grid = {'logisticregression__C': 1/np.logspace(-3,4,num=8),
                 'logisticregression__penalty': ['l1', 'l2']}

    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score,greater_is_better=True),
                        cv=skf, return_train_score = True, iid=True)
    # do kfold CV on _other
    grid.fit(X_other, y_other)
    #print(grid.cv_results_)
    return grid, X_test, y_test 

In [32]:
log_grid, log_X_test, log_y_test = ML_log(X,y,42,4)
print(log_grid.best_score_)
print(log_grid.score(log_X_test,log_y_test))
print(log_grid.best_params_)

0.8372948766128022
0.8369660528623325
{'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}


In [33]:
import pickle
file = open('loggrid.save', 'wb')
pickle.dump((log_grid, log_X_test,log_y_test),file)
file.close()

In [34]:
rf_grid, rf_X_test, rf_y_test = ML_rf(X,y,42,4)
print(rf_grid.best_score_)
print(rf_grid.score(rf_X_test,rf_y_test))
print(rf_grid.best_params_)

0.8316422397594889
0.8322059376174371
{'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 4}


In [35]:
file = open('rfgrid.save', 'wb')
pickle.dump((rf_grid, rf_X_test,rf_y_test),file)
file.close()

In [38]:
log_test_scores = [log_grid.score(log_X_test,log_y_test)]
log_params = [log_grid.best_params_]
for i in range(1,4):
    grid, X_test, y_test = ML_log(X,y,randstates[i],4)
    test_score = grid.score(X_test, y_test)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    log_test_scores.append(test_score)
    log_params.append(grid.best_params_)
    print('Run %d complete' % i)
print('test accuracy:',np.around(np.mean(log_test_scores),4),'+/-',np.around(np.std(log_test_scores),4))
log_idx = np.argmax(log_test_scores)
print('best test run:', log_test_scores[log_idx], log_params[log_idx])

{'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}
best CV score: 0.8372479017913065
test score: 0.8365276211950394
Run 1 complete
{'logisticregression__C': 0.001, 'logisticregression__penalty': 'l2'}
best CV score: 0.8375140924464487
test score: 0.8369660528623325
Run 2 complete
{'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}
best CV score: 0.8378585744707503
test score: 0.8358386571464361
Run 3 complete
test accuracy: 0.8366 +/- 0.0005
best test run: 0.8369660528623325 {'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}


In [52]:
log_test_scores = [0.8369660528623325, 0.8365276211950394, 0.8369660528623325, 0.8358386571464361, 0.8375297507202806]
print(log_test_scores)
print(np.mean(log_test_scores))
print(np.std(log_test_scores))

[0.8369660528623325, 0.8365276211950394, 0.8369660528623325, 0.8358386571464361, 0.8375297507202806]
0.8367656269572843
0.0005621647418844459


In [53]:
rf_test_scores = [rf_grid.score(log_X_test,log_y_test)]
rf_params = [rf_grid.best_params_]
for i in range(1,4):
    grid, X_test, y_test = ML_rf(X,y,randstates[i],4)
    test_score = grid.score(X_test, y_test)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    rf_test_scores.append(test_score)
    rf_params.append(grid.best_params_)
    print('Run %d complete' % i)
print('test accuracy:',np.around(np.mean(rf_test_scores),4),'+/-',np.around(np.std(rf_test_scores),4))
rf_idx = np.argmax(rf_test_scores)
print('best test run:', rf_test_scores[rf_idx], rf_params[rf_idx])

{'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 9}
best CV score: 0.8348365276211951
test score: 0.8360265564324189
Run 1 complete
{'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 6}
best CV score: 0.8326130527370663
test score: 0.8329575347613679
Run 2 complete
{'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 3}
best CV score: 0.8322059376174371
test score: 0.8311411749968683
Run 3 complete
test accuracy: 0.8331 +/- 0.0018
best test run: 0.8360265564324189 {'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 9}


In [57]:
#rf_test_scores.append(0.8315169735688338)
print(np.mean(rf_test_scores))
print(np.std(rf_test_scores))

0.8327696354753853
0.0017425062937797043


In [58]:
(np.mean(rf_test_scores) - 0.8123044185755414) / 0.0013489581914233135

15.171127637581309

In [37]:
(np.mean(log_test_scores) - 0.812351246398597) / np.std(log_test_scores)

88.33333333334365

In [ ]:
# {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l2'}
# best CV score: 0.8376393586371038
# test score: 0.8375297507202806
# Run 0 complete
# {'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}
# best CV score: 0.8372479017913065
# test score: 0.8365276211950394
# Run 1 complete
# {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l2'}
# best CV score: 0.8375140924464487
# test score: 0.8369660528623325
# Run 2 complete
# {'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}
# best CV score: 0.8378585744707503
# test score: 0.8358386571464361
# Run 3 complete

In [ ]:
# {'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 2}
# best CV score: 0.8340379556557685
# test score: 0.8315169735688338
# Run 0 complete
# {'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 9}
# best CV score: 0.8348365276211951
# test score: 0.8360265564324189
# Run 1 complete
# {'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 6}
# best CV score: 0.8326130527370663
# test score: 0.8329575347613679
# Run 2 complete
# {'randomforestclassifier__max_depth': 7, 'randomforestclassifier__min_samples_split': 3}
# best CV score: 0.8322059376174371
# test score: 0.8311411749968683
# Run 3 complete